# Build a Dashboard Application with Plotly Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

- TASK 1: Add a Launch Site Drop-down Input Component
- TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
- TASK 3: Add a Range Slider to Select Payload
- TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

Note:Please take screenshots of the Dashboard and save them. Further upload your notebook to github.

The github url and the screenshots are later required in the presentation slides.

The completed dashboard application should look sthg like this
![Alt text](image.png)

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

- Which site has the largest successful launches?
- Which site has the highest launch success rate?
- Which payload range(s) has the highest launch success rate?
- Which payload range(s) has the lowest launch success rate?
- Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

We have **four different launch sites** and we would like to first see which one has the largest success count.

Then, we would like to select one specific site and check its detailed success rate (class=0 vs. class=1).

As such, we will need a dropdown menu to let us select different launch sites.

Find and complete a commented dcc.Dropdown(id='site-dropdown',...) input with following attributes:
- **id** attribute with value site-dropdown
- **options** attribute is a list of dict-like option objects (with label and value attributes). You can set the **label** and **value** all to be the launch site names in the **spacex_df** and you need to include the default **All** option. 
     -  Example: options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'site1', 'value': 'site1'}, ...]

- **value** attribute with default dropdown value to be ALL meaning all sites are selected
- **placeholder** attribute to show a text description about this input area, such as **Select a Launch Site here**
- **searchable** attribute to be True so we can enter keywords to search launch sites

# IMPORT Libraries and Data

In [1]:
import pandas as pd
import dash # main library used to create web-based applications
# Import the Dash class from the dash library
from dash import Dash, html, dash_table
# import html (short for dash_html_components), this is used for building the layout of components. We use 'html' as acronym
# Import dash_core_components (acronym 'dcc') also used for building components such as dropdown, sliders, checkboxes, buttons, geolocation, graphs ...
from dash import dcc
# import input, output from dash.dependencies so we use them without referring to dash.dependencies
from dash.dependencies import Input
from dash.dependencies import Output
import plotly.express as px # plotly express for creating the figures. dash library is built on top of plotly

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
print(spacex_df.shape)
print(spacex_df.head(5))

(56, 7)
   Unnamed: 0  Flight Number  Launch Site  class  Payload Mass (kg)  \
0           0              1  CCAFS LC-40      0                0.0   
1           1              2  CCAFS LC-40      0                0.0   
2           2              3  CCAFS LC-40      0              525.0   
3           3              4  CCAFS LC-40      0              500.0   
4           4              5  CCAFS LC-40      0              677.0   

  Booster Version Booster Version Category  
0  F9 v1.0  B0003                     v1.0  
1  F9 v1.0  B0004                     v1.0  
2  F9 v1.0  B0005                     v1.0  
3  F9 v1.0  B0006                     v1.0  
4  F9 v1.0  B0007                     v1.0  


In [3]:
print('Launch Site Names', spacex_df['Launch Site'].unique())
print('Number of Launch Sites', len(spacex_df['Launch Site'].unique())) # so there are 4 launch sites
max_payload = spacex_df['Payload Mass (kg)'].max()
print(max_payload) # 9600
min_payload = spacex_df['Payload Mass (kg)'].min()
print(min_payload) # 0
# Since min is 0 and max is 9,600 so range of payload is 0:10,000

Launch Site Names ['CCAFS LC-40' 'VAFB SLC-4E' 'KSC LC-39A' 'CCAFS SLC-40']
Number of Launch Sites 4
9600.0
0.0


In [7]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                             value='ALL',
                                             placeholder="State",
                                             searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 100: '100'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 3:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):    
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, 
                     values='class', 
                     names='Launch Site', 
                     title='Total Success Launches By Site')        
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby('class').count().reset_index()        
        fig = px.pie(filtered_df, 
                     values='Unnamed: 0', 
                     names='class', 
                     title='Total Launches for site {}'.format(entered_site))        
        # return the outcomes piechart for a selected site
    return fig
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value"))
def get_scatter_chart(entered_site, payload_range):
    print('Params: {} {}'.format(entered_site, payload_range))
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= int(payload_range[0])) &
                                (spacex_df['Payload Mass (kg)'] <= int(payload_range[1]))
                               ]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='All sites - payload mass between {:8,d}kg and {:8,d}kg'.format(int(payload_range[0]),int(payload_range[1])))
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == entered_site) & 
                                (spacex_df['Payload Mass (kg)'] >= int(payload_range[0])) &
                                (spacex_df['Payload Mass (kg)'] <= int(payload_range[1]))
                               ]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Site {} - payload mass between {:8,d}kg and {:8,d}kg'.format(entered_site,int(payload_range[0]),int(payload_range[1])))
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

Params: ALL [0, 9600]
Params: CCAFS LC-40 [0, 9600]
Params: CCAFS LC-40 [1000, 10000]
Params: CCAFS LC-40 [3000, 10000]
Params: CCAFS LC-40 [5000, 10000]
Params: CCAFS LC-40 [6000, 10000]
Params: CCAFS LC-40 [2000, 10000]
Params: CCAFS LC-40 [1000, 10000]
Params: CCAFS LC-40 [6000, 10000]
Params: CCAFS LC-40 [3000, 10000]
Params: CCAFS LC-40 [4000, 10000]
Params: CCAFS LC-40 [5000, 10000]
Params: CCAFS LC-40 [6000, 10000]
Params: CCAFS LC-40 [7000, 10000]
Params: CCAFS LC-40 [8000, 10000]
Params: CCAFS LC-40 [9000, 10000]
Params: CCAFS LC-40 [100, 10000]
Params: CCAFS LC-40 [100, 7000]
Params: CCAFS LC-40 [100, 4000]
Params: CCAFS LC-40 [1000, 4000]


## Additional

In [5]:
def update_pie_chart(selected_site):
    # 'selected_site' is a parameter in the function 'update_pie_chart' here
    # Filter the dataframe based on the selected launch site
    # this is if user chose to see success rates of all sites in 1 pie chart
    if selected_site == 'ALL':
        filtered_df = spacex_df.copy() # then don't filter, use all the dataframe
        title = 'Total Successful Launches for All Sites'
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site] # this is if user chose only the rows for the filtered data when its the site you want 
        title = f'Total Successful Launches for {selected_site}'
    
    # Create the pie chart for success rate (called 'class')
    pie_chart = px.pie(filtered_df,
                       names='class', # specify class as input for pie chart (either 0,1)
                       title=title
                       )
    return pie_chart


In [6]:
update_pie_chart('CCAFS LC-40')